In [ ]:
from influxdb import InfluxDBClient
import pandas as pd
from influxdb import DataFrameClient
import pytz, datetime, time
from datetime import timedelta, datetime

In [ ]:
from tqdm import tqdm

In [ ]:
influxClient = DataFrameClient(host='influxdb.demo.svc.cluster.local', port=8086, #username='root', password='root', 
                           ssl=False, verify_ssl=False, database='loadtesting')

In [ ]:
influxLTClientResultSet = influxClient.query("select * from lt_result")

In [ ]:
influxLT_df = influxLTClientResultSet['lt_result']

In [ ]:
pd.set_option('display.max_rows', 5000)
pd.pandas.set_option('display.max_columns', None)

In [ ]:
import pytz
local_tz = pytz.timezone('America/Toronto') # Set local timezone for InfluxDB based times calculations
today=datetime.today().strftime('%Y-%m-%d')

In [ ]:
influxLT_df.head(10)

In [ ]:
influxLT_df.index.dtype

In [ ]:
influxLT_df.shape

In [ ]:
influxLT_df.index.name = 'RecordDateTime'

In [ ]:
influxLT_df.to_csv('lt_data_influx_2024-01-15.csv',header=True,)

In [ ]:
influxLT_df[influxLT_df.index >'2023-09-13 00:00:00' ].shape

In [ ]:
i = influxLT_df.index[5]
influxLT_df.loc[i].to_json("row{}.json".format(i))

In [ ]:
record_as_dict = influxLT_df.iloc[6].to_dict()

In [ ]:
import json
with open('result-long-lt.json', 'w') as fp:
    json.dump(record_as_dict, fp)

In [ ]:
influxLT_df_with_spikes = influxLT_df[influxLT_df.index >'2023-09-12 00:00:00' ]

In [ ]:
influxLT_df_no_spikes = influxLT_df[influxLT_df.index <'2023-09-12 00:00:00']

In [ ]:
influxLT_df_no_spikes.shape

In [ ]:
# influxLT_df_no_spikes has prometheus data only with wiremock
# influxLT_df_with_spikes has prometheus data with wiremock and load-generator

In [ ]:
from prometheus_api_client import PrometheusConnect, MetricSnapshotDataFrame, MetricRangeDataFrame, MetricsList, Metric
from prometheus_api_client.utils import parse_datetime

In [ ]:
prom = PrometheusConnect(url="http://prometheus-operated.demo.svc.cluster.local:9090",
                         disable_ssl=True)

In [ ]:
q_lg_msg_w_spike_10s_rate = 'rate(load_generator_total_msg[10s]) + rate(load_generator_spike_cycle_total_msg[10s])'

In [ ]:
q_lg_msg_10s_rate = 'rate(load_generator_total_msg[10s])'

In [ ]:
q_lg_spike_msg_10s_rate = 'rate(load_generator_spike_cycle_total_msg[10s])'

In [ ]:
q_lg_avg_latency = 'load_generator_avg_latency_ms'

In [ ]:
q_lg_avg_latecy_10s_rate = 'rate(load_generator_avg_latency_ms[10s])'

In [ ]:
def convert_prometheus_to_df(prom_metric_data):
    df_values = pd.DataFrame(prom_metric_data[0]['values'], columns=['epoch','value'])
    df_values['EventDateTime'] = df_values['epoch'].apply(datetime.fromtimestamp)
    df_values.set_index('EventDateTime', inplace=True)
    df_values["value"] = df_values["value"].apply(lambda x: format(float(x),".2f"))
    
    return df_values
    

In [ ]:
def extract_prometheus_data(influxDF, query,fileName, step="10s"):
    for i in tqdm(range(influxDF.shape[0])):
        ltStartTS = round(influxDF.iloc[i]['ltStartTS'])
        ltStopTS = round(influxDF.iloc[i]['ltStopTS'])
        startTime = datetime.fromtimestamp(ltStartTS)
        stopTime = datetime.fromtimestamp(ltStopTS)
        prom_metric_data =  prom.custom_query_range(query=query,
                                      start_time=startTime,
                                      end_time=stopTime,
                                      step=step)
        if len(prom_metric_data) > 0:
            df_values = convert_prometheus_to_df(prom_metric_data)
            csv_name=f'{fileName}_{ltStartTS}_{ltStopTS}.csv'
            df_values.to_csv(csv_name, index=True, index_label='EventDateTime', header=True)
        else:
            print(f'Empty prometheus metric set at index {i}\n')

In [ ]:
influxLT_df_with_spikes_prom = influxLT_df_with_spikes.iloc[7:]

In [ ]:
influxLT_df_with_spikes_prom.shape

In [ ]:
q_wmock_total_req_rate_10s = 'sum(rate(wiremock_requests_total[10s]))'

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_wmock_total_req_rate_10s, 'wireMock10sMSGRate')

In [ ]:
q_wmock_overall_service_cpu_load_10s_rate = 'sum(avg by (service) (rate(process_cpu_seconds_total{service=~"wiremock-metrics-.*"}[10s])*100))'

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_wmock_overall_service_cpu_load_10s_rate, 'wireMock10sOverallServiceCPULoadRate')

In [ ]:
q_wiremock_overall_jvm_thread_10s_rate = 'sum(rate(jvm_threads_current{service=~"wiremock-metrics-.*"}[10s]))'

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_wiremock_overall_jvm_thread_10s_rate, 'wireMock10sOverallJVMThreadRate')

In [ ]:
q_wmock_overall_heapKB_10s_rate = 'sum(rate(jvm_memory_bytes_used{area="heap",service=~"wiremock-metrics-.*"}[10s])/1024)'

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_wmock_overall_heapKB_10s_rate, 'wireMock10sOverallHeapKBRate')

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_lg_msg_w_spike_10s_rate, 'loadGen10sMSGwSpikesRate')

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_lg_msg_10s_rate, 'loadGen10sMSGRate')

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_lg_spike_msg_10s_rate, 'loadGen10sSpikeMSGRate')

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_lg_avg_latency, 'loadGenAvgLatencyMS5s', step="5s")

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_lg_avg_latency, 'loadGenAvgLatencyMS10s', step="10s")

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_lg_avg_latency, 'loadGenAvgLatencyMS1s', step="1s")

In [ ]:
extract_prometheus_data(influxLT_df_with_spikes, q_lg_avg_latecy_10s_rate, 'loadGenAvgLatencyMSRate10s', step="10s")